In [1]:
from tqdm import tqdm
from OmniEvent.infer import infer, get_pretrained
import torch
import sys
import io

# 全局变量来存储模型
model = None
tokenizer = None

def efficient_infer(texts, task="EE"):
    global model
    global tokenizer
    if model is None:
        ed_model, ed_tokenizer = get_pretrained("s2s-mt5-ed", 'cuda')  # 加载模型
        eae_model, eae_tokenizer = get_pretrained("s2s-mt5-eae", 'cuda')
        model = [ed_model, eae_model]
        tokenizer = [ed_tokenizer, eae_tokenizer]
    
    # 创建一个临时的字符串流来捕获所有输出
    temp_stdout = io.StringIO()
    
    results = []
    for text in tqdm(texts, desc="Processing Texts"):
        # 保存当前的标准输出
        old_stdout = sys.stdout
        # 重定向标准输出到临时流
        sys.stdout = temp_stdout
        
        # 调用infer函数，此时其内部的print调用不会输出到控制台
        result = infer(text=text, model=model, tokenizer=tokenizer, task=task)
        
        # 恢复标准输出
        sys.stdout = old_stdout
        
        results.append(result)
    
    # 清空临时输出
    temp_stdout.close()
    
    return results

texts = [
    "2022年北京市举办了冬奥会。",
    "今日有网友爆料：“在湖北襄阳，因家里起了点小争执...",
    "据报道，昨日在上海一科技公司发生了数据泄露事件。美国总统在国会的一次演讲中提到了即将到来的经济改革。",
    "美国总统在国会的一次演讲中提到了即将到来的经济改革。",
    "近日，一起严重的森林火灾在加利福尼亚北部爆发。",
    "昨天晚上，东京经历了数十年来最大的一次地震。",
    "本周早些时候，一位名人在社交媒体上宣布了其即将发布的新书。",
    "在巴黎举行的时装周上，多位设计师展示了他们的最新作品。",
    "国际奥委会宣布将在2032年把夏季奥运会带到悉尼。",
    "昨日，一名科学家团队在瑞士宣布了一项突破性的医学研究成果。",
    "近日，网络安全问题再次引起了全球范围内的关注和讨论。",
    "在昨晚的奖项典礼上，一位年轻音乐家获得了最佳新人奖。",
    "有消息称，一家大型制药公司将投资数亿美元用于疾病研究。",
    "教育部门宣布将增加资金支持远程教育项目。",
    "国际环保组织在最新报告中强调了气候变化的严峻挑战。",
    "警方昨日在多伦多市中心进行了一次大规模的毒品搜查行动。",
    "一位著名电影导演在昨晚的访谈中透露了他的下一部电影计划。",
    "最新市场研究报告显示，电动汽车销量在过去一年中大幅上涨。",
    "昨天，一项关于全球经济趋势的研究在伦敦的一个国际会议上被发布。",
    "在最近的一次科技展览上，一家初创公司展示了一种创新的人工智能应用。"
]
results = efficient_infer(texts)

/usr/local/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


load from local file: /root/.cache/OmniEvent_Model/s2s-mt5-ed model
load from local file: /root/.cache/OmniEvent_Model/s2s-mt5-ed tokenizer


/usr/local/miniconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


load from local file: /root/.cache/OmniEvent_Model/s2s-mt5-eae model
load from local file: /root/.cache/OmniEvent_Model/s2s-mt5-eae tokenizer


Processing Texts: 100%|██████████| 20/20 [00:17<00:00,  1.15it/s]


In [1]:
import numpy as np
import pandas as pd
import re
path = "./data/train_data_en.csv"
train_data_en = pd.read_csv(path)
def remove_urls(text):
    # Regular expression to match URLs
    pattern = r'https?://\S+|www\.\S+'
    return re.sub(pattern, '', text)

#Apply the function to each element in the 'content' column
train_data_en['content'] = train_data_en['content'].apply(remove_urls)
texts = train_data_en['content'].tolist()

In [1]:
import numpy as np
import pandas as pd
import re
import pickle

# 读取 .pkl 文件
with open('./hy-tmp/val_new_cleaned.pkl', 'rb') as f:
    train_data_en = pickle.load(f)
def remove_urls(text):
    # Regular expression to match URLs
    pattern = r'https?://\S+|www\.\S+'
    return re.sub(pattern, '', text)
train_data_en['content'] = train_data_en['content'].apply(remove_urls)
texts = train_data_en['content'].tolist()

In [2]:
len(texts)

1654

In [4]:
#尝试并行
import sys
import io
import os
from tqdm import tqdm
import torch
from OmniEvent.infer import infer, get_pretrained
import pickle

os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3,4,5'
# 重新初始化 CUDA 运行时（PyTorch 示例，如果是 TensorFlow 可能需要重启 Kernel）
torch.cuda.init()
# 全局变量来存储模型和分词器
models = None
tokenizers = None

def setup_model():
    global models
    global tokenizers
    if models is None:
        # 加载模型
        ed_model, ed_tokenizer = get_pretrained("s2s-mt5-ed", 'cuda')
        eae_model, eae_tokenizer = get_pretrained("s2s-mt5-eae", 'cuda')

        # 确保模型在 CUDA 上
        ed_model = ed_model.cuda()
        eae_model = eae_model.cuda()
        
        # 使用 DataParallel 封装模型
        ed_model = torch.nn.DataParallel(ed_model)
        eae_model = torch.nn.DataParallel(eae_model)
        
        models = [ed_model, eae_model]
        tokenizers = [ed_tokenizer, eae_tokenizer]

def efficient_infer(texts, task="EE"):
    setup_model()
    
    # 创建一个临时的字符串流来捕获所有输出
    temp_stdout = io.StringIO()
    
    results = []
    for text in tqdm(texts, desc="Processing Texts"):
        # 保存当前的标准输出
        old_stdout = sys.stdout
        # 重定向标准输出到临时流
        sys.stdout = temp_stdout
        # 使用适当的模型和分词器
        result = infer(text=text, model=[models[0].module, models[1].module], tokenizer=tokenizers, task=task, device = 'cuda')
        
        # 恢复标准输出
        sys.stdout = old_stdout
        
        results.append(result)
    
    # 清空临时输出
    temp_stdout.close()
    
    return results

results = efficient_infer(texts)
with open('./fake_news_event_val', 'wb') as f:
    pickle.dump(results , f)

load from local file: /root/.cache/OmniEvent_Model/s2s-mt5-ed model
load from local file: /root/.cache/OmniEvent_Model/s2s-mt5-ed tokenizer


/usr/local/miniconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


load from local file: /root/.cache/OmniEvent_Model/s2s-mt5-eae model
load from local file: /root/.cache/OmniEvent_Model/s2s-mt5-eae tokenizer


Processing Texts: 100%|██████████| 1654/1654 [26:38<00:00,  1.03it/s]


In [5]:
len(results)

1654

In [6]:
#with open('./en_fake_news_event_train', 'rb') as f:
   # results = pickle.load(f)

In [7]:
results[583][0]

{'text': '"We would like to help prepare young people for their important roles in the future, as health emergencies and disease outbreaks may become even more common"-@DrTedros #COVID19 #YouthDay \xa0…',
 'events': []}

In [8]:
import nltk
from nltk.tokenize import word_tokenize

# 假设 result 是你提到的字典列表

# 遍历列表中的每个字典
for item in results:
    # 使用 nltk 的 word_tokenize 方法分割单词
    words = word_tokenize(item[0]['text'])
    # 检查词的数量
    if len(words) > 600:
        # 如果词的数量大于500，截断到前500个词
        item[0]['text'] = ' '.join(words[:600])

# 打印修改后的列表

In [9]:
import nltk
from nltk.tokenize import word_tokenize

# 假设 result 是你提到的字典列表

# 遍历列表中的每个字典
for i in range(len(texts)):
    # 使用 nltk 的 word_tokenize 方法分割单词
    words = word_tokenize(texts[i])
    # 检查词的数量
    if len(words) > 600:
        # 如果词的数量大于500，截断到前500个词
        texts[i] = ' '.join(words[:600])

In [10]:
import pickle
with open('./en_fake_news_event_val', 'wb') as f:
    pickle.dump(results , f)

In [11]:
from translate import Translator
from tqdm import tqdm
def is_english(text):
    """
    检查文本是否主要为英文。要求至少有2个英文字母且不含汉字。
    
    参数:
    text (str): 需要检查的文本。
    
    返回:
    bool: 如果文本主要为英文，则返回True，否则返回False。
    """
    #english_count = sum(char.isalpha() for char in text)  # 计算文本中英文字母的数量
    #contains_chinese = any('\u4e00' <= char <= '\u9fff' for char in text)  # 检查是否含有汉字
    
    return False #english_count >= 2 and not contains_chinese
def format_event_extractions(data_list, sup_translator, sep=" "):
    """
    根据给定的事件提取数据列表，生成格式化的字符串并用指定分隔符连接。
    自动翻译英文为中文。
    """
    if not data_list:
        return ""

    formatted_strings = []
    
    for data in data_list:
        type_str = data['type']
        # 如果type为英文，翻译为中文
        if is_english(type_str):
            type_str = sup_translator.translate(type_str)

        formatted_str = "[CLS] "
        formatted_str += f"【类型】{type_str}【/类型】"
        for arg in data['arguments']:
            mention = arg['mention']
            role = arg['role']
            # 如果mention为英文，翻译为中文
            # 如果role为英文，翻译为中文
            if is_english(role):
                role = sup_translator.translate(role)
            formatted_str += f"【{role}】{mention}【/{role}】"
        trigger = data['trigger']
        formatted_str += f"【触发词】{trigger}【/触发词】。 [SEP]"
        formatted_strings.append(formatted_str)

    return sep.join(formatted_strings)

def format_event_extractions_en(data_list, sep=" "):
    """
    Generate formatted strings based on a given list of event extraction data,
    and concatenate them using the specified separator. This function is tailored
    for handling English event data.
    """
    if not data_list:
        return ""

    formatted_strings = []
    
    for data in data_list:
        type_str = data['type']

        # Initialize the formatted string with [CLS] for starting the sequence
        formatted_str = "[CLS] "
        formatted_str += f"[Type] {type_str} [/Type]"

        # Process each argument in the event
        for arg in data['arguments']:
            mention = arg['mention']
            role = arg['role']

            formatted_str += f"[{role}] {mention} [/{role}]"

        # Append the trigger word at the end
        trigger = data['trigger']
        formatted_str += f"[Trigger] {trigger} [/Trigger]. [SEP]"
        formatted_strings.append(formatted_str)

    return sep.join(formatted_strings)

def process_event_data(event_data, sup_translator, mode, sep=" "):
    """
    处理包含多个样本的事件数据，每个样本包含一个事件列表。

    参数:
    event_data (list): 包含事件数据的大列表，其中每个元素是一个字典，带有一个'events'键。
    sep (str): 用于连接事件格式化字符串的分隔符。

    返回:
    list: 每个样本处理后得到的格式化字符串列表。
    """
    formatted_results = []
    if mode == "en":
    # 遍历大数据集中的每个样本
        for sample in tqdm(event_data, desc="Language Checking"):
        # 获取每个样本中的事件列表
            events = sample[0]['events']
        # 格式化当前样本的事件列表
            formatted_text = format_event_extractions_en(events, sep=sep)
        # 将格式化后的文本添加到结果列表
            formatted_results.append(formatted_text)
    else: 
        for sample in tqdm(event_data, desc="Language Checking"):
        # 获取每个样本中的事件列表
            events = sample[0]['events']
        # 格式化当前样本的事件列表
            formatted_text = format_event_extractions(events, sup_translator, sep=sep)
        # 将格式化后的文本添加到结果列表
            formatted_results.append(formatted_text)
    formatted_results = [[item] for item in formatted_results]
    return formatted_results

# 示例使用
translator = Translator(from_lang='en', to_lang='zh')
event_data = results
mode = 'en' #英文
# 调用函数并打印结果
results_data = process_event_data(event_data, translator, mode, sep=" || ")

Language Checking: 100%|██████████| 1654/1654 [00:00<00:00, 306800.76it/s]


In [12]:
import json
import requests

obj = {"str": "到底发生了什么飞机要绕回来？[泪] 所以是军方通报了之后马来西亚才扩大搜救范围的吗？。"}
req_str = json.dumps(obj).encode()

url = "https://texsmart.qq.com/api"
r = requests.post(url, data=req_str)
r.encoding = "utf-8"
#print(r.text)
res = json.loads(r.text)['entity_list']
print(res)

[{'str': '飞机', 'hit': [7, 2, 7, 2], 'type': {'name': 'basic.tool.physical', 'i18n': '工具', 'flag': 2, 'path': '/product.generic/'}, 'meaning': {'related': ['遥控玩具', '拼图玩具', '智力拼装玩具', '桌面玩具', '木质玩具', '仿真模型', '电子电动玩具', '木制纸品玩具', '布毛绒塑胶玩具', '积木玩具']}, 'tag': 'basic.tool.physical', 'tag_i18n': '工具'}, {'str': '马来西亚', 'hit': [28, 4, 27, 4], 'type': {'name': 'loc.country_region', 'i18n': '国家或地区', 'flag': 1, 'path': '/loc.generic/loc.geo/loc.geo.district/loc.geo.populated_place/'}, 'meaning': {'related': ['莱索托王国', '马拉维共和国', '塞拉利昂共和国', '斯威士兰王国', '冈比亚共和国', '塞舌尔共和国', '乌干达共和国', '坦桑尼亚联合共和国', '莫桑比克共和国', '加纳共和国']}, 'tag': 'loc.country_region', 'tag_i18n': '国家或地区'}]


In [13]:
import requests
import json
from tqdm import tqdm  # 导入tqdm

# 函数化请求过程，以便可以处理多个文本
def call_texsmart_api(texts):
    url = "https://texsmart.qq.com/api"
    results = []
    
    # 遍历所有文本，并使用tqdm显示进度条
    for text in tqdm(texts, desc="Processing Texts"):
        obj = {"str": text}
        req_str = json.dumps(obj).encode()  # 编码请求数据
        response = requests.post(url, data=req_str)  # 发送请求
        response.encoding = "utf-8"  # 设置响应编码
        
        if response.status_code == 200:
            res = json.loads(response.text)['entity_list']  # 解析响应
            results.append(res)  # 添加解析结果到列表
        else:
            kk = None
            print("Failed to process text:")
            results.append(kk)  # 处理失败时添加None
    
    return results

def extract_and_join_strings(entities, sep=' [SEP] '):
    """
    提取entities中每个字典的'str'字段并用指定的分隔符连接成一个字符串。

    参数:
    entities (list): 包含实体字典的列表。
    sep (str): 用于连接字符串的分隔符。

    返回:
    str: 所有'str'字段连接后的字符串。
    """
    # 使用列表推导式提取每个实体中的'str'字段
    strings = [entity['str'] for entity in entities if 'str' in entity]
    
    # 使用sep连接提取出的字符串
    return sep.join(strings)

def process_entity_lists(entity_lists, sep=' [SEP] '):
    """
    处理包含多个实体列表的集合，为每个列表生成格式化的字符串，每个字符串作为单独的列表项返回，并显示进度条。

    参数:
    entity_lists (list): 包含多个实体列表的列表。
    sep (str): 用于连接字符串的分隔符。

    返回:
    list: 每个实体列表处理后得到的包含一个字符串的列表。
    """
    formatted_results = []
    
    # 使用 tqdm 进行循环，显示进度条
    for entities in tqdm(entity_lists, desc="Formatting Entities"):
        # 格式化当前实体列表并添加到结果列表，每个结果作为一个单元素列表
        if entities is not None:
            formatted_text = extract_and_join_strings(entities, sep=sep)
        else:
            formatted_text = ''
        formatted_results.append([formatted_text])
    return formatted_results
    
# 示例使用
# 调用函数处理多个文本
entity_lists = call_texsmart_api(texts)
result_entities = process_entity_lists(entity_lists)


Formatting Entities: 100%|██████████| 1654/1654 [00:00<00:00, 30823.07it/s]


In [81]:
len(result_entities)

1652

In [82]:
#合并

def merge_event_data(results_data, result_entities):
    """
    将两个二维列表中的字符串进行合并，同时显示进度条。

    参数:
    results_data (list): 包含字符串列表的二维列表。
    result_entities (list): 包含字符串列表的二维列表。

    返回:
    list: 合并后的二维列表，其中每个元素是合并后的字符串。
    """
    # 合并两个列表中的字符串，使用tqdm显示进度条
    results_event = [[data[0] + entities[0]] for data, entities in tqdm(zip(results_data, result_entities), total=len(results_data), desc="Merging Events")]
    for i in range(len(results_event)):
        if results_event[i] == ['']:  # 检查是否为空字符串的列表
            results_event[i] = [' [SEP] '] 
    return results_event




# 调用函数并打印结果
results_event = merge_event_data(results_data, result_entities)



texts_dataset = train_data_en
texts_dataset['event'] = results_event

Merging Events: 100%|██████████| 1652/1652 [00:00<00:00, 350444.58it/s]


In [83]:
texts_dataset.head(20)

,content,category,label,event
0,"To find longevity as an artist, one must have ...",gossipcop,0,[[CLS] [Type] attack [/Type][entity] Justin [/...
1,Who is the next Bachelorette 2018? ...it's Bec...,gossipcop,0,[[CLS] [Type] startposition [/Type][position] ...
2,It’s Gilmore Girls meets Dawson’s Creek on Dis...,gossipcop,0,[[CLS] [Type] transport [/Type][origin] Transy...
3,"In actuality, it's unlikely that their bliss i...",gossipcop,1,[[CLS] [Type] marry [/Type][person] their [/pe...
4,(Excerpt) Read more at: E! Online\n\nWake Up T...,gossipcop,0,[[CLS] [Type] meet [/Type][entity] you [/entit...
5,"\n\nPrincess Diana in 1995 with her sons, Prin...",gossipcop,0,[[CLS] [Type] die [/Type][person] Harry [/pers...
6,Biggest Loser trainer and TODAY show health co...,gossipcop,0,[[CLS] [Type] die [/Type][entity] TODAY [/enti...
7,Don't mess with Lupita Nyong'o's hair!\n\nOn T...,gossipcop,0,[[CLS] [Type] broadcast [/Type][entity] Nyong'...
8,Brie Bella is revealing her struggles over whe...,gossipcop,0,[[CLS] [Type] transferownership [/Type][entity...
9,Get the latest from TODAY Sign up for our news...,gossipcop,0,[[CLS] [Type] beborn [/Type][Trigger] birth [/...


In [76]:
texts_dataset.to_csv('/en_fake_news_test.csv', index=False)

In [ ]:
##################
def extract_and_join_strings(entities, sep="SEP"):
    """
    提取entities中每个字典的'str'字段并用指定的分隔符连接成一个字符串。

    参数:
    entities (list): 包含实体字典的列表。
    sep (str): 用于连接字符串的分隔符。

    返回:
    str: 所有'str'字段连接后的字符串。
    """
    # 使用列表推导式提取每个实体中的'str'字段
    strings = [entity['str'] for entity in entities if 'str' in entity]
    
    # 使用sep连接提取出的字符串
    return sep.join(strings)
joined_strings = extract_and_join_strings(entities)
print(joined_strings)

In [16]:
#####################
def extract_strings_from_dicts(dict_list):
    """
    从字典列表中提取 'str' 键的值并组装成列表。

    参数:
    dict_list (list): 包含字典的列表。

    返回:
    list: 包含所有 'str' 值的列表。
    """
    # 初始化结果列表
    result_list = []

    # 遍历列表中的每个字典
    for item in dict_list:
        # 检查 'str' 键是否在字典中
        if 'str' in item:
            # 添加 'str' 键的值到结果列表
            result_list.append(item['str'])

    # 返回结果列表
    return result_list
result_list = extract_strings_from_dicts(res)

In [17]:
result_list 

['飞机', '马来西亚']